In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

model = ChatOpenAI(
    model="llama-3.3-70b-versatile",
    base_url="https://api.groq.com/openai/v1",
    temperature=0
)

print(model.invoke("Say hello").content)


Hello. How can I assist you today?


In [2]:
from langchain.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies two numbers."""
    return a * b

@tool
def divide(a: int, b: int) -> float:
    """Divides two numbers."""
    return a / b

@tool
def add(a: int, b: int) -> int:
    """Adds two numbers."""
    return a + b

tools = [multiply, divide, add]
tools_by_name = {tool.name: tool for tool in tools}
model_with_tools = model.bind_tools(tools)

In [3]:
from langchain.messages import AnyMessage
from typing_extensions import TypedDict, Annotated
import operator

class MessagesState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]
    llm_calls: int

In [5]:
from langchain.messages import SystemMessage

def llm_call(state: dict):
    return {
        "messages":[
            model_with_tools.invoke(
                [
                    SystemMessage(
                        content="You are a helpful assistant tasked with performing arithmetic on a set of inputs."
                    )
                ]
                + state["messages"]
            )
        ],
        "llm_calls": state.get('llm_calls', 0) + 1
    }

In [7]:
from langchain.messages import ToolMessage

def tool_node(state: dict):
    result = []

    for tool_call in state["messages"][-1].tool_calls:
        tool = tools_by_name[tool_call["name"]]
        observation = tool.invoke(tool_call["args"])
        result.append(ToolMessage(
            content=observation,
            tool_call_id=tool_call["id"]
        ))

    return {"messages": result}

In [9]:
from typing import Literal
from langgraph.graph import StateGraph, START, END

def should_continue(state: MessagesState) -> Literal["tool_node", END]:
    """Decide if we should continue the loop or stop based upon whether the LLM made a tool call"""

    messages = state["messages"]
    last_message = messages[-1]

    # If the LLM makes a tool call, then perform an action
    if last_message.tool_calls:
        return "tool_node"

    # Otherwise, we stop (reply to the user)
    return END

In [21]:
agent_builder = StateGraph(MessagesState)

agent_builder.add_node("llm_call", llm_call)
agent_builder.add_node("tool_node", tool_node)

agent_builder.add_edge(START, "llm_call")
agent_builder.add_conditional_edges(
    "llm_call",
    should_continue,
    ["tool_node", END]
)
agent_builder.add_edge("tool_node", "llm_call")

agent = agent_builder.compile()

from langchain_core.runnables.graph import MermaidDrawMethod
from IPython.display import display, Image
import nest_asyncio
nest_asyncio.apply()

# display(
#     Image(
#         agent.get_graph(xray=True).draw_mermaid_png(
#             draw_method=MermaidDrawMethod.PYPPETEER
#         )
#     )
# )

print(agent.get_graph(xray=True).draw_mermaid())

from langchain.messages import HumanMessage
messages = [HumanMessage(content="What is 12 multiplied by 7, then divided by 3, and finally add 10?")]
messages = agent.invoke({"messages" : messages})
for m in messages["messages"]:
    m.pretty_print()


---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	llm_call(llm_call)
	tool_node(tool_node)
	__end__([<p>__end__</p>]):::last
	__start__ --> llm_call;
	llm_call -.-> __end__;
	llm_call -.-> tool_node;
	tool_node --> llm_call;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

================================ Human Message =================================

What is 12 multiplied by 7, then divided by 3, and finally add 10?
================================== Ai Message ==================================
Tool Calls:
  multiply (w39y2hb74)
 Call ID: w39y2hb74
  Args:
    a: 12
    b: 7
  divide (nttjb35r2)
 Call ID: nttjb35r2
  Args:
    a: 84
    b: 3
  add (5wp0ggfha)
 Call ID: 5wp0ggfha
  Args:
    a: 28
    b: 10
================================= Tool Message =================================

84
================================= Tool Message =================================

28.0
===